# Exploring Some Fundamentals


## A Bit of Context

### High School

Back when I was young, I discovered computers and programming with some high school
friends that were making wonders with the help of a french copy of the
[PC Bible](https://www.amazon.fr/Pc-Bible-Knorr/dp/0201883546) book.

More specifically, part of their coding was done with an obscure language they
called *assembly*.

I was still fresh from BASIC and learning Pascal (with Turbo Pascal 6 then 7,
anyone? ;-) ). Including x86 assembly in such development environments was easy,
so I rapidly learned some of it, and discovered its counterparts: registers, bus,
ports, cache, RAM, microcode, cycles, etc.

I do not code in assembly anymore. But what I learned at that time still serves
me today. In my opinion, having a reasonable model of how computers work when
programming is underrated.

### Engineering Shool

A few years later, when I was studying Computer Science, some serious theoritical
background came along, with terms like [Von Neumann architecture](https://en.wikipedia.org/wiki/Von_Neumann_architecture), or [Turing machine](https://en.wikipedia.org/wiki/Turing_machine).

In those years, I was also supposed to learn about Lambda Calculus, Church *et
al.* theoritical work on calculability. I think I completely missed the point at
that time, as I was rapidly lost when trying to "glue" this knowledge with
anything I could relate to what I knew about computers.

For a reason: I had never exposed to any functional programming/reasonning before.

### Back to Present

Fast-forward... I've now coded in a functional fashion with Swift, Kotlin, or Rust,
had several looks into Haskell and Elm, and I'm leaning towards functional style or
patterns in any language that does not prevent me from doing so, such as Python.

So when I found this video: https://www.youtube.com/watch?v=5C6sv7-eTKg, I
thought it was a good time to return to functional programming ~~assembly~~
fundamentals.

What I found interesting in this video:

 * It uses Python, a "usual" (at least to me, i.e. non-functional) language
   to illustrate the various steps.
 
 * It spares me from decrypting the oh-so-obscure mathematical notations that
   pop up in a vast majority of the papers supposed to explain thoses concepts.
   
Both those "qualities" remove cognitive load that were as many barriers back then.

This post is my most recent attempt at understanding lambda-calculus and *Y-combinator*.

One more thing: David Beazley, the video speaker, is bright, clever and very
educational. He also acknowledges that this topic is mind-bending, and I think I
needed that.

## Let's dive into the Maelström

### The Rules

Beazley starts by setting a few simple rules, stating what is allowed and
what is not, while trying to build the various concepts we need.

Those rules are quite simple:
    
1. Everything is a function, nothing else.

2. All functions have a single argument (that is function, remember?).

3. Their return values are be functions, obviously.

He also provides some sound advice, like:
    
> When lost (like I have been), remember. Think of the various concepts as "behaviours".

HTH!

### Boolean Values

> How would you define booleans with such rules?

Ouch...

For starters, let's break a few rules, by using non-function arguments. To
be a bit more concrete, we will consider electrical levels (engineering!)
as Python strings.

For example:

In [ ]:
HIGH = "5V"
LOW = "GND"  # i.e. ground, 0 Volts.

Now we want something that returns `"5V"` when *TRUE*, and `"GND"` when *FALSE*.

We could start like to go with something like:

```python
assert TRUE("5V", "GND") == "5V"
```

and

```python
assert FALSE("5V", "GND") == "GND"
```

Minimal code to achieve this could be:

In [ ]:
def TRUE(left, right):
    return left


def FALSE(left, right):
    return right

Does it run as expected?

Well, it seems so:

In [ ]:
assert TRUE("5V", "GND") == "5V"
assert FALSE("5V", "GND") == "GND"

Now let's try to get back into our initial rules. At least 2 rules are broken here:

* we want single argument functions (rule-2).

* we want functions as arguments (rule-1).

To address the first broken rule, we'll use a simple trick that seems to be called *currying* (https://en.wikipedia.org/wiki/Currying).

The principe is split a multiple-arguments function into several single-argument functions
called in cascade. Instead of writing `f(a, b, c)`, we will code `f(a)(b)(c)` and expect the
same return value.

The principle is to make non-terminal function return another function that will in turn be called with the argument. In Python, it means converting:

In [ ]:
def add_ko(a, b):  # Two args here, rule-2 is broken!
    return a + b

to this:

In [ ]:
def add_ok(a):  # Single arg, OK for rule-2.
    
    def f(b):  # Single arg, OK too for rule-2.
        return a + b

    return f

`add_ok(a)` return the `f` function, that is called with the `b` argument.

Does it work?

Let Python tells us what he thinks about this:

In [ ]:
# Rule-2 broken!
assert add_ko(3, 5) == 8

# Rule-2 compliant \o/.
assert add_ok(3)(5) == 8

With this, we alter initial `TRUE` and `FALSE` definition.

In [ ]:
def TRUE(left):
    def f(right):
        return left
    return f


def FALSE(left):
    def f(right):
        return right
    return f

assert TRUE("5V")("GND") == "5V"
assert FALSE("5V")("GND") == "GND"

This code is now conformant to the "single argument" rule-2. Great.

Remember that rule-1 is also broken, as previous code uses strings as arguments, instead of functions. But if you look closely to the code, only for the assertion part fails us.

When "legal", single argument functions are provided as the `left` and `right` arguments of `TRUE` and `FALSE`,
all our rules are standing.

In other words, our function definitions are OK. It is the calling code that is not.

So we may consider that, at this point, we have booleans.

Well, kind of. Behaviour of booleans, if that helps.

### Boolean Operators

#### Logical Not

Let's start with the simplest boolean operator: `NOT`.

What we want here is easy to state:

```python
assert NOT(TRUE) == FALSE
assert NOT(FALSE) == TRUE
```

Let's start by noticing a few interesting facts about this snippet.

1. `NOT` is a function (rule-1 OK).

2. `NOT` take a single argument (rule-2 OK).

3. Both `TRUE` and `FALSE` are functions, so
    1. `NOT` takes a "legal" function as argument.
    2. `NOT` returns a "legal" function.
   
   That's rule-3!

`NOT` interface is well on tracks. It is now only a matter of finding a suitable
implementation.

To do this, we need to remember that `TRUE` returns the *left*-y argument and
`FALSE` returns *right*-y one.

With our electrical levels, it means:

```python
assert NOT(TRUE)("5V")("GND") == "GND"
assert NOT(FALSE)("5V")("GND") == "5V"
```

How to do this from this reasonable starting point:

```python
def NOT(f):
    return ...  #
```

?

First, notice some ~~funny~~twisted identities:

In [ ]:
assert TRUE(FALSE)(TRUE) == FALSE  # Remember? TRUE returns left.
#             ^---------------^

When using the parameter names, it becomes:

In [ ]:
assert TRUE(left=FALSE)(right=TRUE) == FALSE  # TRUE returns left.
#                  ^---------------------^

and

In [ ]:
assert FALSE(FALSE)(TRUE) == TRUE  # FALSE returns right.
#                     ^--------^

From this "finding", we can parametrize the first part of the identity.

In [ ]:
def NOT(f):
    return f(FALSE)(TRUE)  # Here, f is either TRUE or FALSE, that
    #        ^^^^^  ^^^^     will in turn choose left or right.
    #        left   right

`f` being `TRUE` will chose left function, that is `FALSE`. And conversely.

Is that OK for Python?

In [ ]:
assert NOT(TRUE) is FALSE
assert NOT(FALSE) is TRUE
# `is` instead of `==` is OK here, as TRUE and FALSE are always the same objects.

# Let's get more "physical".
assert NOT(TRUE)("5V")("GND") == "GND"
assert NOT(FALSE)("5V")("GND") == "5V"

We have a `NOT` function/behaviour! \o/

#### Logical And

Similarly, we would like to define a `AND` operator that plays well with our `TRUE` and `FALSE` functions.

Like this:

```python
assert AND(TRUE)(TRUE) is TRUE
assert AND(TRUE)(FALSE) is FALSE
assert AND(FALSE)(TRUE) is FALSE
assert AND(FALSE)(FALSE) is FALSE
```

First remark, `AND` has to deal with 2 arguments. So it will have to look like this:

```python
def AND(x):
    def f(y):
        ...
    return f
```

The point here to think about the binary operator shortcuts that usual programming languages make.

Let me explain: in C or Javascript, when you write `x && y`, what happens depends on `x`.
More specifically, if `x` is `false`, `y` is never evaluated, as the `x && y` will be `false`
anyway.

Let's try to derive an interesting identity to show this:

In [ ]:
assert FALSE("whatever")(FALSE) is FALSE  # Left-y part is bypassed by FALSE.
#                          ^---------^

Conversely, when `x` is `true` in C-ish `x && y`, computation has to reach for `y`
to find out the final value. There are 2 possibilities at this point: either `y`
is `true` and `x && y` is `true`, or `y` is `false`, and so is `x && y`. It means
the result **is** the `y` value.

In other (Python) words:

In [ ]:
assert TRUE(TRUE)("whatever") is TRUE  # Right-y part is bypassed by TRUE.
#             ^--------------------^

assert TRUE(FALSE)("whatever") is FALSE  # Right-y part is bypassed by TRUE.
#             ^--------------------^

From this, we can infer that our `x` (which is either `TRUE` or `FALSE`) should come first, then call our `y` as its `left` argument and leave `right` as the `FALSE` value.

Still there?

Bear with me.

Let's write this down in Python:

In [ ]:
def AND(x):
    def f(y):
        return x(y)(FALSE)

    return f

In [ ]:
assert AND(TRUE)(TRUE) is TRUE
assert AND(TRUE)(FALSE) is FALSE
assert AND(FALSE)(TRUE) is FALSE
assert AND(FALSE)(FALSE) is FALSE

Nice.

We can do even better.

The remaining `FALSE` is called only when `x` is `FALSE`. So we can replace it in the implementation, without changing the outcome:

In [ ]:
def AND(x):
    def f(y):
        return x(y)(x)

    return f

In [ ]:
assert AND(TRUE)(TRUE) is TRUE
assert AND(TRUE)(FALSE) is FALSE
assert AND(FALSE)(TRUE) is FALSE
assert AND(FALSE)(FALSE) is FALSE

#### Logical Or

The implemetation is similar to `AND`'s one. Intuitively, we use the same
trick: we mimic the shortcut taken by usual C-ish `x || y`. In this case,
if `x` is `TRUE`, `y` does not need to be evaluated.

In [ ]:
assert TRUE(TRUE)("anything") is TRUE

If `x` is `FALSE`, `x || y` value is up to `y`.

In [ ]:
assert FALSE("anything")(TRUE) is TRUE
assert FALSE("anything")(FALSE) is FALSE

Possible implementation:

In [ ]:
def OR(x):
    def f(y):
        return x(x)(y)

    return f

If you are lost, let me reformulate `OR` behaviour once again:

* If `x` is `TRUE`, left function (`x` as `TRUE`) is returned.
* If `x` is `FALSE`, let `y` on the right (that can be either `TRUE` or `FALSE`) decide.


In [ ]:
assert OR(TRUE)(TRUE) is TRUE
assert OR(TRUE)(FALSE) is TRUE
assert OR(FALSE)(TRUE) is TRUE
assert OR(FALSE)(FALSE) is FALSE

We have boolean logic (well, boolean logic behaviour at least)!

### Some Python Syntax

Did you know that the initial rules we chose for this journey are leading us
towards [Lambda Calculus](https://en.wikipedia.org/wiki/Lambda_calculus)?

> Lambda calculus (also written as λ-calculus) is a formal system in
> mathematical logic for expressing computation based on function abstraction
> and application using variable binding and substitution. It is a universal
> model of computation that can be used to simulate any Turing machine.
>
> It was introduced by the mathematician Alonzo Church in the 1930s as part
> of his research into the foundations of mathematics.

If you've done a bit of Python, you have already made the link with `lambda`
Python keyword.

It is a way to declare functions. More precisely, *anonymous functions*.

It means there are 2 ways of declaring functions in Python.

In [ ]:
def add(a, b):
    return a + b

assert add(3, 5) == 8

or

In [ ]:
lambda a, b: a + b

assert (lambda a, b: a + b)(3, 5) == 8

2 ways of declaring, but same usage and same result.

The second way is not very convenient, so we give it a name with a usual Python assignment. It is not anonymous anymore!

In [ ]:
add = lambda a, b: a + b

assert add(3, 5) == 8

Python `lambda` functions are somewhat limited, as their definitions is limited to a single expression. But in our case, it is also a convenient way to shorten our code.

Especially when you want only functions with a single argument, as

In [ ]:
def add(a):
    def f(b):
        return a + b
    return f

becomes first:

In [ ]:
def add(a):
    return lambda b: a + b

and finally:

In [ ]:
add = lambda a: lambda b: a + b

Let's rewrite our previous work with this syntax:

In [ ]:
TRUE = lambda x: lambda y: x
FALSE = lambda x: lambda y: y

NOT = lambda f: f(FALSE)(TRUE)

AND = lambda x: lambda y: x(y)(x)
OR = lambda x: lambda y: x(x)(y)

And ensure that all is working as expected:

In [ ]:
assert TRUE("5V")("GND") == "5V"
assert FALSE("5V")("GND") == "GND"

assert NOT(TRUE) is FALSE
assert NOT(FALSE) is TRUE

assert NOT(TRUE)("5V")("GND") == "GND"
assert NOT(FALSE)("5V")("GND") == "5V"

assert AND(TRUE)(TRUE) is TRUE
assert AND(TRUE)(FALSE) is FALSE
assert AND(FALSE)(TRUE) is FALSE
assert AND(FALSE)(FALSE) is FALSE

assert OR(TRUE)(TRUE) is TRUE
assert OR(TRUE)(FALSE) is TRUE
assert OR(FALSE)(TRUE) is TRUE
assert OR(FALSE)(FALSE) is FALSE

Same result, shorter code.

It may not seem easier to read at first, but this is only the beginning of
our journey. You will get used to it rapidly.

We can now go on!

### Numbers

#### Numeration

As we only have function calls, a way to deal with numbers is to count how many times a function is applied.

The function that is called several times is not the point of this principle, so we'll abstract it by making it a argument of our numbers.

For the sake of clarity, we will call this function `f`, and define numbers this way:

In [ ]:
ONE = lambda f: lambda x: f(x)
TWO = lambda f: lambda x: f(f(x))
THREE = lambda f: lambda x: f(f(f(x)))

Just like booleans, we will illustrate this by choosing a `f` and a `x`.

Here is an example:

In [ ]:
incr = lambda x: x + 1

assert ONE(incr)(0) == 1
assert TWO(incr)(0) == 2
assert THREE(incr)(0) == 3

Here is another one, with a different `f` function and another *initial* `x` value:

In [ ]:
concat = lambda x: "*" + x

assert ONE(concat)("") == "*"
assert TWO(concat)("") == "**"
assert THREE(concat)("") == "***"

#### Exploration

We have numbers, and as they are functions, we may want to try a few things.

In [ ]:
TWO(TWO)(incr)(0)

Multiplication?

Well, could have been, but nope.

In [ ]:
TWO(THREE)(incr)(0)

Exponentiation it is!

In [ ]:
FOUR = lambda f: lambda x: f(f(f(f(x))))

assert FOUR(THREE)(incr)(0) == 3**4

With the same number definition, defining `ZERO` is matter of never calling the `f` function:

In [ ]:
ZERO = lambda f: lambda x: x

assert ZERO(incr)(0) == 0

#### Successor

We have number, but we need a link between them to be able to count.

The first link we are building is the `SUCC` function, that finds the number that follows a given one. You know, like *five* comes just after *four*.

Intuitively, it means our *generic* `f` function is called an additional time.

The function `SUCC` will take a number function as argument, we will call it `n`. This number is the kind we defined previously, i.e. it has an "API" that wants 2 arguments that are `f` and `x`.
 
Let try this:

In [ ]:
def SUCC(n):
    #    ^---------- The number function whose successor is wanted
    return lambda f: lambda x:   f(   n(f)(x)   )
    #      ^^^^number API^^^^^   ^    ^^^^^^^ "old/previous number"
    #                            |
    #                    f applied once more

In short

In [ ]:
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

Does it work?

Let's check:

In [ ]:
assert SUCC(TWO)(incr)(0) == THREE(incr)(0)
assert SUCC(SUCC(TWO))(incr)(0) == FOUR(incr)(0)

### Arithmetics

We have numbers, we want math operations!

First, addition. Adding is taking several times the successor of a base number.

In [ ]:
ADD = lambda x: lambda y: x(SUCC)(y)

In other words, `SUCC` is applied `x`times on top of `y`.

In [ ]:
assert ADD(TWO)(THREE)(incr)(0) == 5
assert ADD(TWO)(THREE)(incr)(0) == ADD(FOUR)(ONE)(incr)(0)

Nice :-)

We are going to leave substraction for later, you will understand why.

Now, multiplication.

The main idea is to do `f` `x` times, then you want to do this `y` times. No need of `SUCC` here.

In [ ]:
MUL = lambda x: lambda y: lambda f: y(x(f))

In [ ]:
assert MUL(FOUR)(THREE)(incr)(0) == 12

Awesome.

Division is mathematically and functionally definable with basic steps,
so it is possible, but outside of the scope of this notebook.

### Data Structures

Our way to programming requires being able to aggregate informations.

And the simplest way to start is to use the good, old lispy operators:

* `CONS` to concatenate two "values", as a *t-uple* of 2 values, i.e. a *couple*.
* `CAR` to get the first value of the couple.
* `CDR` to get the second one.

In [ ]:
CONS = lambda a: lambda b: lambda s: s(a)(b)

To stay compliant with our rules, we can consider that `lambda _: "whatever"` is
a function that "stores" `"whatever"`.

Make `"whatever"` is compliant function and we are good. 

Let's take simple values to begin with.

In [ ]:
p = CONS(2)(3)

We already have functions that allows selection.

Remember, we made an electrical switch at the beginning of our journey.

In [ ]:
assert p(TRUE) == 2
assert p(FALSE) == 3

So we use them to implement the missing missing `CAR` and `CDR` functions:

In [ ]:
CAR = lambda p: p(TRUE)
CDR = lambda p: p(FALSE)

A few assertions will ensure we are correct:

In [ ]:
assert CAR(CONS(2)(3)) == 2
assert CDR(CONS(2)(3)) == 3

At top level, it seems to be limited to 2 values. It is not, composition gives us linked lists.

In [ ]:
assert CAR(CDR(CONS(2)(CONS(3)(4)))) == 3

### Predecessor

We are on the way to define what is required to build the substraction operation.

To do this, we will use *couples*, composed of a number and its predecessor.

How do we find the predecessor? Well, we don't.

We do this the other way around.

The couple is in fact a number and its successor, but in reverse order. Finding the
successor is a solved problem, thanks to the `SUCC` function.

Let's define our couple:

In [ ]:
COUPLE = lambda p: CONS(SUCC(CAR(p)))(CAR(p))

It is a data structure (thanks to `CONS`) composed of a successor of number `p` and the number `p`.

Well, not exactly.

Did you notice the 2 `CAR`s within?

`p` is not a number here.

`p` is supposed to be a `COUPLE`!

How are we supposed to build and use that?

First, we will use "alternative" numbers, that are `COUPLE`s instead of previous number functions.

Second, we are going to use what we already have to do so.

In this new scheme, here is what the *four* number is like:

In [ ]:
FOUR_ = FOUR(COUPLE)(CONS(ZERO)(ZERO))

`CONS(ZERO)(ZERO)` is our initial value.

`COUPLE` applied to `CONS(ZERO)(ZERO)` is `CONS(ONE)(ZERO)`.

`COUPLE` applied to `CONS(ONE)(ZERO)` is `CONS(TWO)(ONE)`.

Do this 4 times, and you have CONS(FOUR)(THREE).

And by definition, `FOUR(COUPLE)` applies `COUPLE` four times.

Wow, strange.

Does it even work?

In [ ]:
assert CAR(FOUR_)(incr)(0) == 4
assert CDR(FOUR_)(incr)(0) == 3

It seems so.

We now have the tool to get the predecessor of any number.

Unconvinced?

Check again:

In [ ]:
assert CAR(FOUR_)(incr)(0) == 4
assert CDR(FOUR_)(incr)(0) == 3  # <-- Looks a lot like 4 predecessor.

That leads us to this (non-obvious at first, at least for me) predecessor function:

In [ ]:
PRED = lambda n: CDR(n(COUPLE)(CONS(ZERO)(ZERO)))

What is happening here?

1. We take the `n` number.
2. We build the `(n, n-1)` couple, by applying `COUPLE` `n` times to
   the `CONS(ZERO)(ZERO)` initial value.
3. And we only keep the second part of the resulting structure.

I'm not even kidding:

In [ ]:
EXP = FOUR(THREE)  # Exponentiation, remember?
assert EXP(incr)(0) == 81
assert PRED(EXP)(incr)(0) == 80

At this point, it is important to notice that the point is about what is possible, not its effectiveness.

This will be even more true when building substraction.

### Substraction

When you have the `PRED` function, substraction definition looks a lot like the addition one.

In [ ]:
SUB = lambda x: lambda y: y(PRED)(x)

In [ ]:
assert SUB(FOUR)(TWO)(incr)(0) == 2

Fantastic!

Highly inefficient, but possible. And fantastic!

### Tests

Now this will start to look like we can actually write a computer program.

Tests!

In other terms, *conditions*, i.e. some kind of `if`-clause, but with functions.

Promising, isn't it?

The simplest test that comes to one's mind when dealing with numbers is testing for zero value.

What we want is a function that returns `FALSE` or `TRUE`, when given a function-number `n`.

To do this, we have to remember that `ZERO(f)(x)` never calls `f`.

In [ ]:
def f(_):
    raise Exception()
    
assert ZERO(f)(0) == 0

On the opposite, any other number-function calls `f` at least once.

So `ZERO(f)("whatever")` will return `"whatever"`. Replace `"whatever"`
with `TRUE` and we have half of what we want.

In [ ]:
assert ZERO(f)(TRUE) == TRUE

To get the other half, notice that whatever the (stricly positive) number
of times the `f` function is called, it should always return `FALSE`.

If only we had a function like `lambda _: FALSE`...

Oh wait!

We got it now:

In [ ]:
IS_ZERO = lambda n: n(lambda _: FALSE)(TRUE)

In [ ]:
assert IS_ZERO(ZERO) == TRUE
assert IS_ZERO(ONE) == FALSE
assert IS_ZERO(THREE) == FALSE

Tada! 🎉

## At last, some code

### Factorial!

Factorial is a mathematical operation that is easy to implement in a recursive way.

$$ n! = n(n-1)(n-2)\cdots (2)(1) $$

In [ ]:
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n - 1)

In [ ]:
assert fact(3) == 6

Indeed, $3! = 6$.

Good, good.

Now, we translate exactly this code into our basic functions:

In [ ]:
FACT = lambda n: IS_ZERO(n) (ONE) (MUL(n)(FACT(PRED(n))))
#                   if FALSE ^^^   ^^^^^^^^^^^^^^^^^^^^^ if TRUE

It is almost part-per-part translation.

If you are still reading at this point, you may find it somewhat elegant.

Does it work?

Well... in theory, yes.

That means no, and we are going to explain why and how to make it work.

But let me state first that our code is correct.

It does not work the way it is intended because of the Python interpreter.

Look:

In [ ]:
try:
    FACT(THREE)
except RecursionError:
    print("Oh no! :lemming_emoji:")

#### Problem

`Recursion error`! Python can not have infinite depth of function call. Usual limit is around 1000 calls.

Ok, but why does it triggers so many calls?

Because Python is an **eager** language: it evaluates all expressions before use, even when they are not actually used.

It makes sense intuitively: Python evaluates the value of each argument before calling a function. Even if the argument is not used inside the function.

The vast majority of industry programming languages are eager. So Python is not alone with this "limitation".

Alternatively, non-eager languages are called **lazy**. Some of them, like Haskell, let you choose between eager and lazy evaluations.

The problem is identified. We now have to look for a solution.

#### Solution

We somehow have to find a way to make Python lazy.

And guess what?

This can be achieved with functions.

*Moar* lambdaaaaaaas!.

Just remember that, at this point, we are breaking the initial rules we gave ourselves.

But we are doing so because of Python, not because we made mistakes.

In a way, we are switching from the *Computer Science* domain to the *Software Engineering* one. Compromises...

To fix the problem, we introduce explicitly lazy versions of `TRUE` and `FALSE` functions:

In [ ]:
LAZY_TRUE = lambda x: lambda y: x()
#                                ^^ This is not allowed by our initial rules.
LAZY_FALSE = lambda x: lambda y: y()
#                                 ^^ Same here.

Then we replace the eager version in our good, old `IS_ZERO` test function.

In [ ]:
IS_ZERO = lambda n: n(lambda _: LAZY_FALSE)(LAZY_TRUE)
#                           here^^^^^^^^^^and^^^^^^^^there

Let's define `FACT` again with this new code.

In [ ]:
FACT = lambda n: IS_ZERO(n)(lambda: ONE)(lambda: MUL(n)(FACT(PRED(n))))

And...

In [ ]:
assert FACT(THREE)(incr)(0) == 6

So pleasing.

We actually succeeded at writing and executing a program only made of single-argument functions.

But in fact, we did not.

### We cheated

What?

Nooooooooo.

What happened?

We defined names.

We use `FACT` to implement `FACT`.

In [ ]:
FACT = lambda n: IS_ZERO(n) (ONE) (MUL(n)(FACT(PRED(n))))
#                                         ^^^^ here

THAT is against the rules.

What are we going to do?

## No References

What we want to achieve is to find out how to define `FACT` without using the `FACT` name.

Our problem is that this name is not a function argument, but a reference to an external pre-existing function (a kind of global). Cheating, I told you.

First, a bit of exploration.

Remember, usual `factorial` can be written like this:

In [ ]:
fact = lambda n: 1 if n == 0 else n * fact(n - 1)

In [ ]:
assert fact(5) == 120

Removing a reference name can start by making it an argument:

In [ ]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))(fact)
#              ^                                 ^         ^^^^
#              |--------------->-----------------|           |
#              |---------------<-----------------------------|

In [ ]:
assert fact(5) == 120

Nothing really fancy here.

What we did is basically:

In [ ]:
assert 2 == (lambda y: y)(2)

At this point, we still have the `fact` argument.

Next step, try substitution:

In [ ]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))(
    #   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    #                       |        ^
    #                       Get this |
    #                       and
    #                       put it here |
    #                       |           v
    # vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
    lambda f: lambda n: 1 if n == 0 else n * f(n - 1)
)

There, no more `fact` argument. How great is that?

Well, it is great if it works.

In [ ]:
assert fact(0) == 1

is a good start.

Unfortunately,

In [ ]:
assert fact(1) == 1

Can you spot the problem?

In the first `n * f(n - 1)`, `n` is an integer (`int`) attempts to
multiply to `f(n - 1)` that should evaluate to another `int`.

Except `f(n - 1)` is a function, not an `int`.

In this case, the `f` argument value is the whole subsituted term,
that takes 2 arguments.

To fix this, the second argument has to be provided.

Remember, the first argument is the function we want to use as argument.
Which is `f`.

New attempt:

In [ ]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1))(
    #                                            ^^^^ here...
    lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1)
    #                                        ^^^^ ... and there.
)

In [ ]:
assert fact(0) == 1
assert fact(5) == 120

This definition is not really beautiful with all these repetitions,
but please do not ignore the rejoicing fact that we can now completely
drop reference to any pre-existing name.

What do I mean?

In [ ]:
assert (lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1))(
    lambda f: lambda n: 1 if n == 0 else n * f(f)(n - 1)
)(5) == 120

See? Recursive definition, but no names.

### Abstract Recursion

Consider our previous Python factorial definition:

In [ ]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))(fact)

Wouldn't it be nice to abstract the business part for the we-deal-with-recursion part?

Something like:

In [ ]:
R = lambda f: lambda n: 1 if n == 0 else n * f(n - 1)

on the one hand, and:

In [ ]:
fact = R(fact)

on the other hand.

Could it be as simple?

Well no.

Yes, this works:

In [ ]:
assert fact(5) == 120

But we're cheating again.

```python
fact = R(fact)
```

is valid Python only if `fact` is defined beforehand.

Reference...

But next idea starts from this attempt.

### Fixed point

If you are still there, congratulations.

My tone may be playful and cheerful, but most of the previous steps
were not obvious to me. We came a long way. And the end of this
journey is arriving soon.

But this last step is not the simplest one.

Bear with me, the end is worth it.

(Note that I can lie, but you won't be sure unless you read on, so
at this point, why not trusting me? Did I ever deceive you?)

It is time to talk about the *fixed-point*.

This is an official (read: *mathematical*) definition:

> "`x` is fixed-point of `f`" means `f(f(x)) == f(x) == x`.

For example, consider $\sqrt{1} = 1$: 1 a fixed-point of the square root function.

Under this definition, `fact = R(fact)` means that `fact` is a fixed point of `R`.



Next steps will look like code, but is not actual, runnable Python.

We will use it though, as it is a good way to illustrate the reasoning.

Take a deep breath and let's go.

We will deal with a function called `Y`, that has a ~~unique~~interesting property.

We suppose that `Y(R)` returns a fixed point of `R`.

That implies, per fixed-point definition, that

```python
        Y(R) = R(Y(R))
```

Applying `R`, i.e. writing `R(...)`, is the same as writing `(lambda x: R(x))(...)`.
So we can write it as:

```python
        Y(R) = (lambda x: R(x))(Y(R))
```

We now want to eliminated what is a recursive call to `Y(R)`, so we use the same trick
than our `fact` implementation, i.e. we repeat the function definition instead of its
name:


```python
        Y(R) = (lambda x: R(x))(lambda x: R(x))
```

Remember that when we did this, it failed miserably, because its arguments are not a single
function argument, but a 2 functions. We solved this by "doubling the call", i.e. using `f(f)`
instead of just `f` in both call sites.

Doing the same, we get:

```python
        Y(R) = (lambda x: R(x(x)))(lambda x: R(x(x)))
```

Next step, we want the `R` to become a argument. We do this by adding a `f` parameter,
that takes the `R` value as argument:

```python
        Y(R) = ( lambda f: (lambda x: f(x(x)))(lambda x: f(x(x))) )(R)
```

Finally, because `R` has become a single argument on both side of the affectation,
we can drop it without changing Y definition:

In [ ]:
Y = lambda f: (lambda x: f(x(x)))(lambda x: f(x(x)))

Please welcome the *Y combinator*, invention (or discovery?) of Haskell B. Curry.

Oh, if you are curious: a *combinator* is a function with no free variables.

Now we can play with it.

If we choose to define:

In [ ]:
R = lambda f: lambda n: 1 if n == 0 else n * f(n - 1)

Then `fact = Y(R)` should be sufficient to get our factorial function back.

Theoritically, it is.

In ~~practice~~Python, it is not.

In [ ]:
try:
    fact = Y(R)
except RecursionError:
    pass

Bitten by Python eager evaluation again!

Previously, we wrapped some intermediate function calls to lambdas.

We will do the same here to let Python give us something useful.

Let it be clear that this is only a Python language and runtime ~~limitation~~requirement.

More specifically:

In [ ]:
# Y=lambda f: (lambda x: f(          x(x)   ))(lambda x: f(          x(x)   ))  # <-- previous
Y = lambda f: (lambda x: f(lambda z: x(x)(z)))(lambda x: f(lambda z: x(x)(z)))  # <-- adapted

In [ ]:
fact = Y(R)

In [ ]:
assert fact(5) == 120

Is our "recursion" abstraction working for anything else but factorial?

The answer is yes!

Here is Fibonacci:

In [ ]:
fib = Y(lambda f: lambda n: 1 if n <= 2 else f(n - 1) + f(n - 2))
assert fib(10) == 55

Here it is. My mind is more than bended. My head exploded.

What about you?

## Conclusion

~~Assembly~~fundamentals of functional programming.

I don't have any idea on how I could make this useful on a day-to-day basis.

I can not even pretend making sense of any of this.

But I went much farther than last time it tried, and I'm convinced it will make me better programmer.

And if I'm wrong, at least perhaps you found this interesting, or even educational.


### Links

 * The video I used as base for this notebook: https://www.youtube.com/watch?v=5C6sv7-eTKg

 * You prefer Javascript instead of Python? Try https://lucasfcosta.com/2018/05/20/Y-The-Most-Beautiful-Idea-in-Computer-Science.html
        
 * *Y-combinator*, in Python:
 
   * https://lptk.github.io/programming/2019/10/15/simple-essence-y-combinator.html
   * https://david.ae/posts/the-z-and-y-combinators-in-python/
  
 * Previous derivations and more, still in Python: https://matt.might.net/articles/python-church-y-combinator/, with a fun realization:
 
  > this post is a proof that the indentation-sensitive constructs in Python are strictly optional